## Generation of Json File for  Mapping for point of sales

Module interaction:
- Environement selection
- Point Of View of the POS device:


### ask for mapping information and generate Json Data

Usual configuration

PAYMENT

           "bizStep": "urn:epcglobal:cbv:bizstep:retail_selling",
           "disposition": "urn:epcglobal:cbv:disp:retail_sold",
           "bizLocation": "urn:epc:id:sgln:FixtureSGLN"
           
RETURN

           "bizStep": "urn:epcglobal:cbv:bizstep:shipping",
           "disposition": "urn:epcglobal:cbv:disp:in_transit",
           "bizLocation": "DC BizLocation"
           
READ ONLY

           "bizStep": "urn:epcglobal:cbv:bizstep:cycle_counting",
           "disposition": "urn:epcglobal:cbv:disp:sellable_not_accessible",
           "bizLocation": "urn:epc:id:sgln:FixtureSGLN"
           
           
Important Note:
The bizLocation for fixture must contains the prefix "urn:epc:id:sgln:"


## Select an environment, POS device

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import deviceModelUtils as deviceModelUtils

import json

In [ ]:
status = True 
myEnv = chooseEnv.choosePremise()
if not myEnv.get('urlDP'):
    print ('Error while selecting the environment')
    status = False

if status:
    myEnv = chooseEnv.chooseDevice(myEnv, deviceTypeSelector='POS')
    deviceId = myEnv['device']['id']

    if not deviceId:
        print ('Error while selecting the device')
        status = False

if status:        
    allPovs = deviceUtils.getPovsLinkedToDevice(myEnv,deviceId)
    povName= ''
    if not allPovs:
        print ("No POV found for device "+deviceId)
        status = False
    elif (len(allPovs) >2):
        print ("multiple POVs found for device "+deviceId)
        status = False
    else: 
        povId = allPovs[0]['id']

In [ ]:
# display choice
if status:
    print("Organization: " + myEnv.get('urlDP') + 'Pov Selected: '+ povId)

In [ ]:
import copy
# not mandatory

from datetime import timezone, datetime

if status:
    ts = int(datetime.now(tz=timezone.utc).timestamp() * 1000)

    templateJson =     {
        "assets": [
            #"povID:POS_MODE"
        ],
        "header": {
            "bizStep": "urn:epcglobal:cbv:bizstep:cycle_counting",
            "disposition": "urn:epcglobal:cbv:disp:sellable_accessible",
            "eventTime": 1,
            "action": "OBSERVE",
            "bizLocation": "urn:epc:id:sgln:fixture"
        }
    }

    myForm = uiUtils.inputWithForm('POS POS_PAYMENT definition',['bizStep','disposition','bizLocation'])
    paymentjson = copy.deepcopy(templateJson)
    paymentjson["assets"].append(povId+":POS_PAYMENT")
    tmp = paymentjson["header"]
    tmp['bizStep'] = myForm.get('bizStep')
    tmp['disposition'] = myForm.get('disposition')
    tmp['bizLocation'] = myForm.get('bizLocation')
    tmp['eventTime'] = ts


    myForm = uiUtils.inputWithForm('POS POS_RETURN definition',['bizStep','disposition','bizLocation'])
    returnjson = copy.deepcopy(templateJson)
    returnjson["assets"].append(povId+":POS_RETURN")
    tmp = returnjson["header"]
    tmp['bizStep'] = myForm.get('bizStep')
    tmp['disposition'] = myForm.get('disposition')
    tmp['bizLocation'] = myForm.get('bizLocation')
    tmp['eventTime'] = ts


    myForm = uiUtils.inputWithForm('POS POS_READ_ONLY" definition',['bizStep','disposition','bizLocation'])
    readonlyjson = copy.deepcopy(templateJson)
    readonlyjson["assets"].append(povId+":POS_READ_ONLY")
    tmp = readonlyjson["header"]
    tmp['bizStep'] = myForm.get('bizStep')
    tmp['disposition'] = myForm.get('disposition')
    tmp['bizLocation'] = myForm.get('bizLocation')
    tmp['eventTime'] = ts


    mappingJson = [paymentjson , returnjson, readonlyjson]

In [ ]:
#%% display
if status and mappingJson :
    windowTitle = "Append HUB -povIdModeToEpcisEventMapping.json-" 
    text = json.dumps(mappingJson,indent=4)
    uiUtils.displayMessage(windowTitle,text)

## SAVE Json File 

In [ ]:
#%% save
if status and mappingJson :
    defaultName = povId + "_povIdModeToEpcisEventMapping.json"
    fname = uiUtils.saveOneFile(".",defaultName,"Select output json file",(("json files","*.json"),("all files","*.*")))
    if fname is None:
        # skip by user
        print('skip by user')
        
    elif (not fname):
        print('skip by user')
        
    else:
        with open(fname, 'w') as outfile:
            json.dump(mappingJson, outfile,indent=4)